In [7]:
import os
import sys
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from random import sample
import math

In [8]:
from DataFormatting import *

In [13]:
training_location = os.path.join('data', '*.cif')
training_files = glob.glob(training_location)

In [14]:
pro_df = protein_data_frame(training_files)
print(pro_df)

        H     O    S  A  SE  B      C  D     N  E  ...  PSU  PYR  TYR  ARG  \
7WWY    0   445   11  0   0  0   1365  0   405  0  ...    0    0    1    4   
5JAW    0  6211  130  0   0  0  21451  0  5690  0  ...    0    0   23   24   
4G6Q    0   504    0  0   0  0   1677  0   469  0  ...    0    0    4   14   
2RJL    0   213    5  0   0  0    745  0   185  0  ...    0    0    7    7   
6ILH    0  1464   55  0   0  0   5100  0  1352  0  ...    0    0   22   27   
...   ...   ...  ... ..  .. ..    ... ..   ... ..  ...  ...  ...  ...  ...   
5DLR    0   693   37  0   0  0   2493  0   675  0  ...    0    0    4   13   
4GUZ    0  1644   20  0   0  0   5696  0  1604  0  ...    0    0    9   23   
3K5T    0  1014   16  0   0  0   3609  0   952  0  ...    0    0   33   45   
6OZ8    0   990   30  0   0  0   3266  0   908  0  ...    0    1    5    8   
1VWB  244   196    2  0   0  0    643  0   176  0  ...    0    0    6    4   

      CSO  trip  doub  sing  OLIG_COUNT  SYMMETRY  
7WWY    0  

In [16]:
pro_df.to_csv('data.csv')